# mean reverting algo
# connected to bybit / futures trading

# libraries

In [9]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import json
import requests
import datetime as dt
import schedule
import time
import math
from binance.client import Client
import os

In [1]:
os.environ['API_KEY'] = 'key'
os.environ['API_SECRET'] = 'secret'

NameError: name 'os' is not defined

# algo

In [11]:
def strategy():
    #data import
    url = 'https://api.binance.com/api/v3/klines'
    params = {'symbol': 'BTCUSDT', 'interval': '1h', 'limit': 800}
    response = requests.get(url, params=params)
    req = json.loads(response.text)
    data = pd.DataFrame(req, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'qav', 'trades',
                                     'taker_base_vol', 'taker_quote_vol', 'ignored'])
    data.drop(columns=['open', 'high', 'low', 'volume', 'close_time', 'qav', 'trades','taker_base_vol', 'taker_quote_vol', 'ignored'], inplace = True)
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    data['close'] = data['close'].astype(float)

    # strategy
    ma = 5
    data['returns'] = np.log(data["close"]).diff()
    data['ma'] = data['close'].rolling(ma).mean()
    data['ratio'] = data['close'] / data['ma']

    # percentiles
    percentiles = [7.01293, 44.356, 41.22, 95.66, 94.750435]
    p = np.percentile(data['ratio'].dropna(), percentiles)

    # entries
    short = p[-1]
    long = p[0]
    data['position'] = np.where(data.ratio > short, -1, np.nan)
    data['position'] = np.where(data.ratio < long, 1, data['position']) 
    data['position'] = data['position'].ffill()
    data['entry'] =  data.position.diff()

    # neutral
    neutral_high = 1.154077131171284
    neutral_low = 0.9366029469329376

    data['position'] = np.where((data.ratio > neutral_high) & (data.position == -1), 0, data['position'])
    data['position'] = np.where((data.ratio < neutral_low) & (data.position == 1), 0, data['position'])
    data['position'] = data['position'].ffill()

    new_position = data['position'][-1]
    
    # binance api
    
    # credentials
    api_key = os.getenv('API_KEY')
    api_secret = os.getenv('API_SECRET')

    # % of the wallet to use in each trade
    porcentaje_saldo = 0.85

    # contract
    symbol = 'BTCUSDT' 
    contract_type = 'PERPETUAL'

    # leverage
    leverage = 10

    # creating an instance in the binance client
    client = Client(api_key, api_secret)

    # obtaining account total balance
    account_info = client.futures_account()
    balances = account_info['assets']

    # calculating balance avialable for trading
    for balance in balances:
        if balance['asset'] == 'USDT':
            total_balance = float(balance['walletBalance'])
            break
    else:
        total_balance = 0.0

    pos_size = total_balance * porcentaje_saldo # 85%

    # margin
    order_size = pos_size * leverage

    # actual position
    position_info = client.futures_position_information(symbol=symbol)
    current_position = float(position_info[0]['positionAmt'])

    # last pair price
    ticker_price = client.futures_ticker(symbol=symbol)

    # quantity to buy/sell
    quantity = math.floor(order_size / float(ticker_price['lastPrice']) * 1000) / 1000

    # closing last position (if not in neutral)
    if current_position > 0 and new_position < 0:  # if there is a long pos, new pos will be short
        order_side = Client.SIDE_SELL
        quantity = abs(current_position)
        order = client.futures_create_order(
            symbol=symbol,
            side=order_side,
            type=Client.FUTURE_ORDER_TYPE_MARKET,
            quantity=quantity,
            newOrderRespType='RESULT',
            recvWindow=60000
        )
        print("closed long position:", order)
        time.sleep(5)  # wait a few secs for the order to complete
        current_position = 0  # update the new position to neutral
    elif current_position < 0 and new_position > 0:  # if there is a short pos, new pos will be long
        order_side = Client.SIDE_BUY
        quantity = abs(current_position)
        order = client.futures_create_order(
            symbol=symbol,
            side=order_side,
            type=Client.FUTURE_ORDER_TYPE_MARKET,
            quantity=quantity,
            newOrderRespType='RESULT',
            recvWindow=60000
        )
        print("closed short position:", order)
        time.sleep(5)  # wait a few secs for the order to complete
        current_position = 0  # update the new position to neutral

    # open a new position if there is none or if the current pos is neutral
    if current_position == 0:
        if new_position > 0:  # if long
            order_side = Client.SIDE_BUY
        else:  # if short
            order_side = Client.SIDE_SELL
        order = client.futures_create_order(
            symbol=symbol,
            side=order_side,
            type=Client.FUTURE_ORDER_TYPE_MARKET,
            quantity=quantity,
            newOrderRespType='RESULT',
            recvWindow=60000
        )
        print("new position opened:", order)

    # obtaining position info afert the order is executed
    position_info = client.futures_position_information(symbol=symbol)
    print(position_info) 

In [12]:
strategy()

BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action, request ip: 163.247.52.14

# execution (hourly)

In [ ]:
schedule.every().hour.at(":00").do(strategy)
schedule.every().hour.at(":01").do(strategy)
while True:
    schedule.run_pending()
    time.sleep(1)

[{'symbol': 'BTCUSDT', 'positionAmt': '-0.002', 'entryPrice': '30575.9', 'markPrice': '30505.21657059', 'unRealizedProfit': '0.14136685', 'liquidationPrice': '35687.93248008', 'leverage': '10', 'maxNotionalValue': '30000000', 'marginType': 'cross', 'isolatedMargin': '0.00000000', 'isAutoAddMargin': 'false', 'positionSide': 'BOTH', 'notional': '-61.01043314', 'isolatedWallet': '0', 'updateTime': 1688215021849}]
[{'symbol': 'BTCUSDT', 'positionAmt': '-0.002', 'entryPrice': '30575.9', 'markPrice': '30509.00000000', 'unRealizedProfit': '0.13380000', 'liquidationPrice': '35687.93248008', 'leverage': '10', 'maxNotionalValue': '30000000', 'marginType': 'cross', 'isolatedMargin': '0.00000000', 'isAutoAddMargin': 'false', 'positionSide': 'BOTH', 'notional': '-61.01800000', 'isolatedWallet': '0', 'updateTime': 1688215021849}]
[{'symbol': 'BTCUSDT', 'positionAmt': '-0.002', 'entryPrice': '30575.9', 'markPrice': '30543.10000000', 'unRealizedProfit': '0.06560000', 'liquidationPrice': '35687.9324800